In [3]:
#!/usr/bin/env python3
import sys
sys.path.insert(0, './src/results')

import numpy as np
import pandas as pd
from load import load_all_p1, load_all_p2
from utils import niceNameModel, niceNamePhn, PHNALL

data = load_all_p2(clear_badlines=True)
print(data.head())

Loading data
Removing bad lines
       user     doc domain   mkb                      model  line  mkbIndex  \
0  brouke-1  brouke  audit  OPPS                  zlabs-nlp    19         0   
1  brouke-1  brouke  audit  OPPS                       OPPO    19         0   
2  brouke-1  brouke  audit  OPPS                        ref    19         0   
3  brouke-1  brouke  audit  OPPS  newstest2020-online-b.sgm    19         0   
4  brouke-1  brouke  audit  OPPS                   Online-G    19         0   

                                             txt_ref  \
0  Dne 11. prosince 2007 schválila Evropská komis...   
1  Dne 11. prosince 2007 schválila Evropská komis...   
2  Dne 11. prosince 2007 schválila Evropská komis...   
3  Dne 11. prosince 2007 schválila Evropská komis...   
4  Dne 11. prosince 2007 schválila Evropská komis...   

                                             txt_tgt lang  ...  disappearance  \
0  Dne 11. prosince 2007 schválila Evropská komis...   cs  ...            N

In [4]:
# data['model'] = data['model'].astype(str)
# data['line'] = data['line'].astype(str)
# data['mkbIndex'] = data['mkbIndex'].astype(str)

In [5]:
dataB0 = data[data['user'] == 'brouke-0']
dataB1 = data[data['user'] == 'brouke-1']
dataJ = dataB0.merge(dataB1, on=['mkbIndex'], suffixes=['_b0', '_b1'])
print(dataJ.head())

    user_b0  doc_b0 domain_b0      mkb_b0      model_b0  line_b0  mkbIndex  \
0  brouke-0  brouke     audit  COMMUNIQUÉ  eTranslation       88         0   
1  brouke-0  brouke     audit  COMMUNIQUÉ  eTranslation       88         0   
2  brouke-0  brouke     audit  COMMUNIQUÉ  eTranslation       88         0   
3  brouke-0  brouke     audit  COMMUNIQUÉ  eTranslation       88         0   
4  brouke-0  brouke     audit  COMMUNIQUÉ  eTranslation       88         0   

  txt_ref_b0      txt_tgt_b0 lang_b0  ...  disappearance_b1  typography_b1  \
0   KOMUNIKÉ  COMMUNIQU. . .      cs  ...               NaN            NaN   
1   KOMUNIKÉ  COMMUNIQU. . .      cs  ...               NaN            NaN   
2   KOMUNIKÉ  COMMUNIQU. . .      cs  ...               NaN            NaN   
3   KOMUNIKÉ  COMMUNIQU. . .      cs  ...               NaN            NaN   
4   KOMUNIKÉ  COMMUNIQU. . .      cs  ...               NaN            NaN   

   style_b1  nontranslated_b1  terminology_b1  inconsistency_b

In [29]:
pairs = [
    ('brouke-0', 'brouke-1'),
    ('kufrc-0', 'kufrc-1'),
    ('autoc-0', 'autoc-1'),
    ('autoc-0', 'autoc-2'),
    ('autoc-2', 'autoc-1'),
    ('broukc-0', 'broukc-1'),
    ('broukc-0', 'broukc-2'),
    ('broukc-2', 'broukc-1'),
    ('euroe-0', 'euroe-3'),
    ('euroe-0', 'euroe-2'),
    ('euroe-2', 'euroe-3'),
    ('euroe-4', 'euroe-3'),
    ('euroe-4', 'euroe-2'),
]

phnAcc = {k:[] for k in PHNALL}
phnKappa = {k:[] for k in PHNALL}
phnCorr = {k:[] for k in PHNALL}
phnCorrZ = {k:[] for k in PHNALL}

for k0, k1 in pairs:
    dataB0 = data[data['user'] == k0]
    dataB1 = data[data['user'] == k1]
    dataJ = dataB0.merge(dataB1, on=['mkbIndex', 'line', 'model', 'mkb'], suffixes=['_b0', '_b1'])
    for phn in PHNALL:
        b0pos = np.isnan(dataJ[phn + '_b0']) == False
        b1pos = np.isnan(dataJ[phn + '_b1']) == False
        chanceHit = (b0pos.sum()*b1pos.sum() + (len(b0pos) -  b0pos.sum())*(len(b1pos) -  b1pos.sum()))/(len(b0pos)*len(b1pos))

        hit = b0pos == b1pos
        phnAcc[phn].append(hit.sum() / len(hit))
        if chanceHit != 1:
            kappa = (hit.sum()/b0pos.count() - chanceHit)/(1-chanceHit)
            phnKappa[phn].append(kappa)

        dataB = dataJ[b0pos & b1pos]
        if len(dataB) > 1:
            b0val = list(dataB[phn+'_b0'])
            b1val = list(dataB[phn+'_b1'])
            
            avg = np.average(b0val + b1val)
            if any([x != avg for x in (b0val+b1val)]):
                corr = np.corrcoef(b0val, b1val)[0][1]
            else:
                corr = 1
            if not np.isnan(corr):
                phnCorr[phn].append(corr)
        elif len(dataB) == 1:
            phnCorr[phn].append(1)
        
        
        dataB = dataJ
        if len(dataB) > 1:
            vals = zip(dataB[phn+'_b0'], dataB[phn+'_b1'])
            
            vals = [(x[0], 0) if np.isnan(x[1]) else (0, x[1]) if np.isnan(x[0]) else x for x in vals if (not np.isnan(x[0]) or not np.isnan(x[1]))]
            b0val = [x[0] for x in vals]
            b1val = [x[1] for x in vals]
            
            avg = np.average(b0val + b1val)
            if any([x != avg for x in (b0val+b1val)]):
                corr = np.corrcoef(b0val, b1val)[0][1]
            else:
                corr = 1
            if not np.isnan(corr):
                phnCorrZ[phn].append(corr)
        elif len(dataB) == 1:
            phnCorrZ[phn].append(1)
            
#         print(f'{phn:<15} {hit.sum() / hit.count():.3f}    {b0pos.sum():>5}/{b1pos.sum():>5} pos')

print("Pairwise comparison")
print("Phenomenon & IAA & Kappa & Corr. & Corr.+ \\\\ \\hline")
for phn in PHNALL:
    acc = np.average(phnAcc[phn])
    kappa = np.average(phnKappa[phn])
    corr = np.average(phnCorr[phn])
    corrz = np.average(phnCorrZ[phn])
    print(f'{niceNamePhn(phn):<15}&{acc:.2f} & {kappa:.2f} &  {corr:.2f} & {corrz:.2f} \\\\')

/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/vilda/.local/lib/python3.8/site-

/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/vilda/.local/lib/python3.8/site-pa

Pairwise comparison
Phenomenon & IAA & Kappa & Corr. & Corr.+ \\ \hline
Disappearance  &0.90 & 0.43 &  0.52 & 0.06 \\
Typography     &0.95 & 0.20 &  0.55 & -0.13 \\
Sense          &0.91 & 0.17 &  0.73 & -0.09 \\
Style          &0.94 & 0.24 &  1.00 & 0.19 \\
Terminology    &0.90 & 0.41 &  0.07 & -0.03 \\
Inconsistency  &0.88 & 0.13 &  0.18 & -0.08 \\
Non-translated &0.94 & 0.20 &  0.64 & 0.30 \\
Conflicting    &0.77 & 0.02 &  1.00 & 0.62 \\
Other grammar  &0.96 & 0.10 &  1.00 & -0.35 \\
Semantic role  &0.97 & -0.01 &  nan & 0.43 \\
Over-translated&0.98 & 0.37 &  1.00 & 1.00 \\


/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/vilda/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/vilda/.local/lib/python3.8/site-packages/numpy/core/_meth